## Modules

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

In [9]:
mini_instance="mini_instance.xlsx"
instance_simple="instance_WPY_simple.xlsx"
instance_realiste="instance_WPY_realiste_jalon1.xlsx"

## Mini insatnce:

In [11]:
import pandas as pd

def excel_to_dict(file):
    # Load the Excel file
    xl = pd.ExcelFile(file)
    
    # Get the names of the sheets
    sheet_names = xl.sheet_names
    
    # Initialize a dictionary to store the data
    data_dict = {}
    
    # Iterate over each sheet
    for sheet_name in sheet_names:
        # Read the sheet into a pandas DataFrame
        df = pd.read_excel(file, sheet_name=sheet_name)
        
        # Convert the DataFrame into a list of dictionaries and add to the data dictionary
        data_dict[sheet_name] = df.to_dict(orient='records')
    
    return data_dict



# Convert the Excel file to a dictionary
data_dictionary = excel_to_dict(mini_instance)

# You can now use data_dictionary as needed
print(data_dictionary)


{'Chantiers': [{'Chantier': 'WPY_REC', 'Nombre de voies': 3, 'Indisponibilites': 0}, {'Chantier': 'WPY_FOR', 'Nombre de voies': 3, 'Indisponibilites': 0}, {'Chantier': 'WPY_DEP', 'Nombre de voies': 3, 'Indisponibilites': 0}], 'Machines': [{'Machine': 'DEB', 'Type de tache': 'Débranchement', 'Duree ': 15, 'Indisponibilites': 0}, {'Machine': 'FOR', 'Type de tache': 'Formation', 'Duree ': 15, 'Indisponibilites': '(6,13:00-13:00);(7, 13:00-13:00)'}, {'Machine': 'DEG', 'Type de tache': 'Dégarage', 'Duree ': 15, 'Indisponibilites': 0}], 'Sillons arrivee': [{'n°TRAIN': 'sillon1', 'HARR': datetime.time(9, 0), 'JARR': '02/05/2023'}, {'n°TRAIN': 'sillon2', 'HARR': datetime.time(13, 0), 'JARR': '02/05/2023'}, {'n°TRAIN': 'sillon3', 'HARR': datetime.time(16, 0), 'JARR': '02/05/2023'}], 'Sillons depart': [{'n°TRAIN': 'sillon4', 'HDEP': datetime.time(21, 0), 'JDEP': Timestamp('2023-05-02 00:00:00')}, {'n°TRAIN': 'sillon5', 'HDEP': datetime.time(21, 0), 'JDEP': Timestamp('2023-05-02 00:00:00')}, {'n°